Sylvan LE DEUNFF

# Projet Traitement et Analyse d’Images
# IMR3 – 2019-2020
# Classification d’images


---

Aujourd'hui la croissance du numérique, et le développement des services Cloud amènent une explosion du nombre de documents multimédias à stocker. En particulier les images qui sont omniprésentes sur les réseaux sociaux comme Facebook, Instagram ou Snapchat. Pour pouvoir rechercher ces images ou en extraire de l'information, on peut utiliser des métadonnées, mais celles-ci sont bien souvent incomplètes ou erronnées. Il devient donc nécessaire de pouvoir classifier ces images non-plus selon leur métadonnées, mais en se basant sur leur contenu.

Pour ce TP, nous nous intéressons à une problématique de classification d'images. L'objectif ici est donc d'être en mesure d'attribuer une étiquette (classe) à une image à partir de son contenu. Pour cela nous comparerons 2 approches :

- la classification basée sur des vecteurs de distances entre les primitives de l'image.
- la classification par un réseau de neurones convolutif.



## Préparation des données

Ici les images étudiées proviennent de la base de données Corel. C'est un ensemble d'image provenant d'une galerie photo de même nom, qui ont été labellisés et rangées dans des répertoires portant les noms de ces étiquettes.

*Exemple de structure de la BDD*
```
coreldb
├── ...
├── pet_cat
└── pet_dog
```

### Répartition des données en 3 sous-ensembles

Avant de nous attaquer à un problème de classification d'images, il convient pour chaque feature de répartir les données en 3 sous-ensembles. A savoir :

* **apprentissage**: ce sont les données qui seront utilisées pour entraîner notre modèle.

* **validation**: les données de cet ensemble ne sont pas utilisées lors de l'apprentissage. elles permettront d'ajuster les hyper-paramètres du modèle.

* **test**: ces données servent à évaluer la précision d'un modèle *après entrainement*. Il est essentiel qu'elles n'aient pas servi pendant les phases d'apprentissage ou de test. Dans le cas contraire, on risque de sur-entrainer notre modèle qui ne fera plus à proprement parler des "prédictions" mais de la mémorisation.

|Ensemble|Proportion|
|-|-|
|Apprentissage|70%|
|Validation|15%|
|Test|15%|






In [ ]:
import os, shutil
from random import shuffle

COREL_DB_PATH = 'coreldb'
WORKSPACE_DB = 'database'

def random_classes(n):
  """
  Tire au maximum n classes aléatoires parmi celles contenues dans la base COREL.
  """
  # liste les fichiers disponibles dans le répertoire de la base COREL
  files = os.listdir(COREL_DB_PATH)

  classes = list(filter(lambda file: (
      # conserve uniquement les classes (répertoires)
      os.path.isdir(os.path.join(COREL_DB_PATH, file))
      # ignore les repertoire caches (systeme) sous linux
      and not file.startswith('.')
  ), files))

  # melange (en place) la liste de classes
  shuffle(classes)

  # retourne les n (au plus) premieres classes
  return classes[:n]
  

def split_classes_data(classes):
  """
  Reparti pour chaque classe les images en 3 sous-repertoires.
  Proportion: train=70% validation=15% test=15%
  """
  try:
    # supprime la base de tests precendente si elle existe
    shutil.rmtree(WORKSPACE_DB)
  except FileNotFoundError:
    pass

  # recrée un répertoire pour la base de tests
  os.mkdir(WORKSPACE_DB)

  # crée 3 répertoires pour répartir nos classes
  folders = ('train', 'validation', 'test')
  for folder in folders:
    os.mkdir(os.path.join(WORKSPACE_DB, folder))
  
  # pour chacune des classes choisies
  for classe in classes:
    # recupere les images disponibles dans la base COREL
    images = os.listdir(os.path.join(COREL_DB_PATH, classe))
    shuffle(images)
    # calcule en fonction du nombre d'image les indices des slices
    n = len(images)
    indexes = [0, int(0.7*n), int(0.85*n), n]
    
    for k in range(3):
      # cree le repertoire de la classe dans chaque sous-repertoire
      os.mkdir(os.path.join(WORKSPACE_DB, folders[k], classe))
      # déplace les images de la base COREL vers la base de travail
      for image in images[indexes[k]: indexes[k+1]]:
        shutil.copy(
            os.path.join(COREL_DB_PATH, classe, image),
            os.path.join(WORKSPACE_DB, folders[k], classe, image)
        )

classes = random_classes(2)
split_classes_data(classes)

Après exécution de ce script la base de travail générée est la suivante.

```
database/
├── test
│   ├── pet_cat
│   └── pet_dog
├── train
│   ├── pet_cat
│   └── pet_dog
└── validation
    ├── pet_cat
    └── pet_dog
```

## Classification d'images basée sur leurs attributs (CBIR)

Nous allons à présent chercher à classifier des images en fonctions de leurs attributs. Pour cela nous nous appuyons sur la librairie suivante :

https://github.com/pochih/CBIR/

## Classification par un CNN

Un réseau de neurones convolutif (CNN) est un algorithme d'apprentissage profond qui peut prendre une image d'entrée, attribuer une importance à divers aspects/objets de l'image et être capable de les différencier les uns des autres. Le prétraitement requis dans un CNN est beaucoup moins important que dans d'autres algorithmes de classification. En effet les méthodes basées sur l'analyse des primitives comme le CBIR, nécessitent de concevoir les filtres à la main, ce qui demande des connaissances d'expertise. A l'inverse, les CNN ont la capacité d'apprendre "naturellement" ces filtres/caractéristiques.

L'architecture d'un CNN est analogue à celle du schéma de connectivité des neurones dans le cerveau humain et s'inspire de l'organisation du cortex visuel. Les neurones individuels répondent aux stimuli que dans une région restreinte du champ visuel appelée le champ de réception. Un ensemble de ces champs se chevauchent pour couvrir la totalité de la zone visuelle.

![](https://miro.medium.com/max/1255/1*vkQ0hXDaQv57sALXAJquxA.jpeg)

### Préparation des données

![](https://miro.medium.com/max/550/1*15yDvGKV47a0nkf5qLKOOQ.png)

### Structure du NN utilisé

### Entrainement du modèle



### Tests du modèle et évaluation des résultats

Afin d'évaluer les qualité des prédictions effectuées, on définit une fonction :
```
test_model(model, mode, output_images=False)
```
où
- **model**: est un classifieur ayant été entrainé.
- **mode**: type de classification (valeurs possibles: `binary` ou `categorical`)
- **output_images**: défini s'il faut écrire les résultats dans le répertoire `<database>/output`

Cette fonction
- charge les images du répertoire `<database>/test` sous forme d'un générateur
- test le modèle entrainé, avec les images de test

### Paramètres de l'apprentissage

- **batch_size** détermine le nombre d'échantillons dans chaque mini lot. Son maximum est le nombre de tous les échantillons, ce qui rend la descente de gradient précise, la perte diminuera vers le minimum si le taux d'apprentissage est assez faible, mais les itérations sont plus lentes. Son minimum est de 1, ce qui entraîne une descente stochastique du gradient : Rapide mais la direction du pas du gradient n'est basée que sur un seul exemple, la perte peut sauter. batch_size permet d'ajuster entre les deux extrêmes : direction précise du gradient et itération rapide. De plus, la valeur maximale de batch_size peut être limitée si votre modèle + ensemble de données ne rentre pas dans la mémoire disponible (GPU).

- **steps_per_epoch** le nombre d'itérations par lot avant qu'une époque d'entraînement ne soit considérée comme terminée. Si vous avez un ensemble d'entraînement de taille fixe, vous pouvez l'ignorer mais cela peut être utile si vous avez un ensemble de données énorme ou si vous générez des augmentations de données aléatoires à la volée, c'est-à-dire si votre ensemble d'entraînement a une taille (générée) infinie. Si vous avez le temps de parcourir l'ensemble de votre ensemble de données d'entraînement, je vous recommande de ne pas tenir compte de ce paramètre.

- **validation_steps** similaire à steps_per_epoch mais sur l'ensemble des données de validation au lieu des données d'entraînement. Si vous avez le temps de passer en revue l'ensemble de vos données de validation, je vous recommande de ne pas utiliser ce paramètre.
